In [9]:
pip install pymongo

Note: you may need to restart the kernel to use updated packages.


In [11]:
import pymongo
import csv

client = pymongo.MongoClient("mongodb://localhost:27017/")
db_name = 'Amazon_Product_Warehouse'
db = client[db_name]
collection_name = 'products: Gift Cards'
collection = db[collection_name]

csv_files = ['merge_gift_cards_final.csv']

data_to_insert = []

for file in csv_files:
    with open(file, 'r') as csv_file:
        csv_reader = csv.DictReader(csv_file)

        for row in csv_reader:
            data_to_insert.append(row)

if data_to_insert:
    result = collection.insert_many(data_to_insert)

    if result.inserted_ids:
        print(f"Documents inserted with IDs: {result.inserted_ids}")

Documents inserted with IDs: [ObjectId('65650927d7023f0ecabe85fd'), ObjectId('65650927d7023f0ecabe85fe'), ObjectId('65650927d7023f0ecabe85ff'), ObjectId('65650927d7023f0ecabe8600'), ObjectId('65650927d7023f0ecabe8601'), ObjectId('65650927d7023f0ecabe8602'), ObjectId('65650927d7023f0ecabe8603'), ObjectId('65650927d7023f0ecabe8604'), ObjectId('65650927d7023f0ecabe8605'), ObjectId('65650927d7023f0ecabe8606'), ObjectId('65650927d7023f0ecabe8607'), ObjectId('65650927d7023f0ecabe8608'), ObjectId('65650927d7023f0ecabe8609'), ObjectId('65650927d7023f0ecabe860a'), ObjectId('65650927d7023f0ecabe860b'), ObjectId('65650927d7023f0ecabe860c'), ObjectId('65650927d7023f0ecabe860d'), ObjectId('65650927d7023f0ecabe860e'), ObjectId('65650927d7023f0ecabe860f'), ObjectId('65650927d7023f0ecabe8610'), ObjectId('65650927d7023f0ecabe8611'), ObjectId('65650927d7023f0ecabe8612'), ObjectId('65650927d7023f0ecabe8613'), ObjectId('65650927d7023f0ecabe8614'), ObjectId('65650927d7023f0ecabe8615'), ObjectId('65650927d7

In [13]:
collection_name = 'products: Luxury Items'
collection = db[collection_name]

csv_files = ['merge_lux_final.csv']

data_to_insert = []

for file in csv_files:
    with open(file, 'r') as csv_file:
        csv_reader = csv.DictReader(csv_file)

        for row in csv_reader:
            data_to_insert.append(row)

if data_to_insert:
    result = collection.insert_many(data_to_insert)

    if result.inserted_ids:
        print(f"Documents inserted with IDs: {result.inserted_ids}")

Documents inserted with IDs: [ObjectId('6565094bd7023f0ecabedd44'), ObjectId('6565094bd7023f0ecabedd45'), ObjectId('6565094bd7023f0ecabedd46'), ObjectId('6565094bd7023f0ecabedd47'), ObjectId('6565094bd7023f0ecabedd48'), ObjectId('6565094bd7023f0ecabedd49'), ObjectId('6565094bd7023f0ecabedd4a'), ObjectId('6565094bd7023f0ecabedd4b'), ObjectId('6565094bd7023f0ecabedd4c'), ObjectId('6565094bd7023f0ecabedd4d'), ObjectId('6565094bd7023f0ecabedd4e'), ObjectId('6565094bd7023f0ecabedd4f'), ObjectId('6565094bd7023f0ecabedd50'), ObjectId('6565094bd7023f0ecabedd51'), ObjectId('6565094bd7023f0ecabedd52'), ObjectId('6565094bd7023f0ecabedd53'), ObjectId('6565094bd7023f0ecabedd54'), ObjectId('6565094bd7023f0ecabedd55'), ObjectId('6565094bd7023f0ecabedd56'), ObjectId('6565094bd7023f0ecabedd57'), ObjectId('6565094bd7023f0ecabedd58'), ObjectId('6565094bd7023f0ecabedd59'), ObjectId('6565094bd7023f0ecabedd5a'), ObjectId('6565094bd7023f0ecabedd5b'), ObjectId('6565094bd7023f0ecabedd5c'), ObjectId('6565094bd7

In [14]:
print("Databases:", client.list_database_names())

Databases: ['Amazon_Product_Warehouse', 'admin', 'config', 'local', 'yelp']


In [15]:
db = client["Amazon_Product_Warehouse"]
print("Collections:", db.list_collection_names())

Collections: ['products: Gift Cards', 'products: Luxury Items']


In [16]:
gift_cards_collection = db['products: Gift Cards']
luxury_items_collection = db['products: Luxury Items']

In [17]:
query = {"overall": {"$gte": 4}}

print(f"Total documents in 'products: Luxury Items' collection: {luxury_items_collection.count_documents({})}")

results = luxury_items_collection.find(query)

print(f"Query being executed: {query}")

for item in results:
    print(item)

Total documents in 'products: Luxury Items' collection: 60780
Query being executed: {'overall': {'$gte': 4}}


In [18]:
#aggregation pipeline
#Top Reviewers for Luxury Items:
result_cursor = luxury_items_collection.aggregate([
    {"$group": {"_id": "$reviewerID", "totalReviews": {"$sum": 1}}},
    {"$sort": {"totalReviews": -1}},
    {"$limit": 5}
])
for result in result_cursor:
    print(result)

{'_id': 'A3091RP0SPZLMN', 'totalReviews': 213}
{'_id': 'A2V5R832QCSOMX', 'totalReviews': 213}
{'_id': 'AQY5XBYSENNZQ', 'totalReviews': 201}
{'_id': 'A2OW1FKQFPIA5D', 'totalReviews': 192}
{'_id': 'AHN86VFJIJ2JP', 'totalReviews': 189}


In [22]:
#Number of Verified vs. Non-Verified Reviews:
result_cursor = luxury_items_collection.aggregate([
    {
        "$group": {
            "_id": "$verified",
            "count": {"$sum": 1}
        }
    },
    {
        "$project": {
            "verification_status": {
                "$cond": {
                    "if": {"$eq": ["$_id", True]},
                    "then": "Verified",
                    "else": "Non-Verified"
                }
            },
            "count": 1,
            "_id": 0
        }
    },
    {
        "$group": {
            "_id": "$verification_status",
            "total_count": {"$sum": "$count"}
        }
    }
])

for result in result_cursor:
    print(result)


{'_id': 'Non-Verified', 'total_count': 60780}


In [20]:
#Most Voted Gift Cards Reviews:
top_reviews = gift_cards_collection.find().sort("vote", -1).limit(5)
number = 1

for review in top_reviews:
    print(f"Serial number: {number}")
    print(f"Title: {review['title']}")
    print(f"ID: {review['_id']}")
    print(f"Number of Votes: {review['vote']}")
    print("\n")
    number += 1


Serial number: 1
Title: Noodles &amp; Company Gift Card
ID: 6561be78b1b06a913b88b8ae
Number of Votes: 9


Serial number: 2
Title: JCPenney Gift Card
ID: 65650921d7023f0ecabe817e
Number of Votes: 9


Serial number: 3
Title: JCPenney Gift Card
ID: 65650921d7023f0ecabe817b
Number of Votes: 9


Serial number: 4
Title: JCPenney Gift Card
ID: 6561be78b1b06a913b88b8a0
Number of Votes: 9


Serial number: 5
Title: JCPenney Gift Card
ID: 6561be78b1b06a913b88b8a3
Number of Votes: 9


